<a href="https://colab.research.google.com/github/afsarahannan/Speech2Text/blob/main/updated_T5_Summarizer_bn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers #use this only once to download it in your colab environment 
import transformers #access the modules with the prefix 'transformers.module_name'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#The datasets needs to be downloaded everytime you run the colab notebook, 
#for some reason it is not being saved in the environment(might be a shortcoming of using free account) 
!pip install datasets
import datasets
from datasets import load_dataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#allow the notebook access to your drive space to store outputs of the notebook 
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataset_file = '/content/drive/My Drive/bn_12_csv.csv'

dataset = load_dataset('csv', data_files=dataset_file, split='train')

dataset = dataset.train_test_split(test_size=0.1) #90% training, 10% testing

train_dataset = dataset['train']
val_dataset = dataset['test']

In [ ]:
dataset['train']['source']

['"#মানুষ১# : আমাকে আজ দেরি করে থাকতে হবে #মানুষ২# : কেন #মানুষ১# :  কারণ আমার অনেক ডাটা এন্ট্রি বাকি আছে ।"',
 '"#মানুষ১# : ঠিক আছে, আমি এটি সম্পর্কেও ভাবছিলাম না, তবে আমি খুব হতাশ হয়ে পড়েছিলাম এবং, আমি আবার রহিমের সাথে কথা বলতে শেষ করেছি।"',
 '"#মানুষ১# : কেন কাস্তমার আমাদের  সারভিস নিয়ে অখুসি #মানুষ২# তারা অভিযোগ করছে যে আমাদের প্রডাক্ট খুব দামি । "',
 '"#মানুষ১# : কিভাবে আমরা আমাদের পপ্রজেক্টের কাজ তারাতারি শেষ করতে পারি  #মানুষ২# : কাজ ভাগ করার জন্য আমাদের দলে আরও সদস্য দরকার । "',
 '"#মানুষ১# : ক্লায়েন্ট কি চায় তা আমাকে একটি ব্রিফ দিন #মানুষ২# : ক্লায়েন্ট আমাদের প্রডাক্টের একটি প্রেসেন্তেসান চান ।"',
 '"#মানুষ১# : কাস্তমার কখন আমাদের বিল ক্লিয়ার করবে #মানুষ২# : কাস্তমার আমাদের কিছু জানায়নি তবে আমি তাদের একটি আপডেটের জন্য কল করব #মানুষ১# :তারা কি বলে আমাকে জানান ।"',
 '"#মানুষ১# : কিভাবে আমরা আমাদের প্রডাক্টের মান উন্নত করতে পারি #মানুষ২# : আমরা প্রডাক্টের আরও ফিছার অ্যাড করতে পারি ।"',
 '"#মানুষ১# : আজ আমাদের একটি সেলশ মিটিং হবে দুপুরের মধ্যে সবাইকে একটি মিটিং লিঙ্ক পাঠান 

In [ ]:
dataset['train']['target']

['"মানুষ২ বলেছে প্রডাক্টের মান উন্নত করতে প্রডাক্টের আরও ফিছার অ্যাড করতে হবে।"',
 '"মানুষ২ বলেছে দলে আরও সদস্য দরকার ।"',
 '"মানুষ২ বলেছে প্রজেক্টে শেষ করতে আরও এক সপ্তাহ সময় লাগবে।"',
 '" মানুষ১ দেরি করে থাকবে কারণ অনেক ডাটা এন্ট্রি বাকি ।"',
 '" দুপুরে সেলশ মিটিং হবে।" ',
 '"মানুষ১ রহিমের সাথে আর কথা বলিনি।"',
 '" মানুষ২ জানতে চায়ে কাস্তমার কবে বিল ক্লিয়ার করবে ।"',
 '" মানুষ২ বলেছে কাস্তমারের কাছে প্রডাক্টের দাম বেশি ।"',
 '" মানুষ২ বলেছে ক্লায়েন্ট প্রডাক্টের একটি প্রেসেন্তেসান চান ।"']

In [ ]:
!pip install transformers tokenizers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import AutoTokenizer, AutoModel


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

In [ ]:
import pandas as pd
#from transformers import AutoTokenizer

#tokenizer = AutoTokenizer.from_pretrained('XLMRobertaTokenizer')

source_text = val_dataset['source']
target_text = val_dataset['target']

tokenized_source_text = tokenizer(list(source_text), truncation=False, padding=False)
tokenized_target_text = tokenizer(list(target_text), truncation=False, padding=False)

max_source = 0
for item in tokenized_source_text['input_ids']:
    if len(item) > max_source:
        max_source = len(item)

max_target = 0
for item in tokenized_target_text['input_ids']:
    if len(item) > max_target:
        max_target = len(item)

In [ ]:
max_source, max_target 

(36, 18)

In [ ]:
def tokenize(batch):
    tokenized_input = tokenizer(batch['source'], padding='max_length', truncation=True, max_length=max_source)
    tokenized_label = tokenizer(batch['target'], padding='max_length', truncation=True, max_length=max_target)

    tokenized_input['labels'] = tokenized_label['input_ids']

    return tokenized_input


In [ ]:
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

# train_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])
# val_dataset.set_format('numpy', columns=['input_ids', 'attention_mask', 'labels'])

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [ ]:
type(val_dataset)

datasets.arrow_dataset.Dataset

The train(10 rows) and validation(2 rows) dataset is in an arrow data format with the following keys:  

**source** = The main dialogue text  

**target** = The summarization of the texts  

**input_ids** = Includes an array of vectorized representation of the tokenized words, this information is the same as the information stored inside **labels**  

**attention_mask** = includes an array of 1s and 0s for padding 

In [ ]:
from transformers.training_args_tf import TFTrainingArguments
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

#model = T5ForConditionalGeneration.from_pretrained('t5-base') #loading the pretrained model from T5
model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/banglat5")

output_dir = '/content/drive/My Drive/dir' #setting the output directory for where the model will be saved

#this block of code will set the arguments for training the model
training_args = TrainingArguments(    
    output_dir=output_dir, #directory for the model checkpoints 
    num_train_epochs=5, #number of complete iterations over the entire training dataset 
    per_device_train_batch_size=0.5, #the batch size for training on each GPU
    per_device_eval_batch_size=0.5, #the batch size for evaluation on each GPU
    eval_accumulation_steps=1, # number of steps to accumulate gradient # Number of eval steps to keep in GPU (the higher, the mor vRAM used)
    prediction_loss_only=True, # compute only loss and not other metrics as evaluation of the model performance, setting this to true will use less RAM
    learning_rate=0.0000001, #sets the learning rate for the optimizer, a low learning rate will slow down the model and takes longer to converge,  
    #best practice to start with a high learning rate and then gradually decrease as the model training achieves better outputs
    evaluation_strategy='steps', # measure for how often the model will run evaluation during the training, Run evaluation every eval_steps
    save_steps=5, # How often to save a checkpoint
    save_total_limit=1, # Number of maximum checkpoints to save
    remove_unused_columns=True, # Removes useless columns from the dataset
    run_name='run_name', # Wandb run name
    logging_steps=5, # How often to log loss to wandb
    eval_steps=5, # How often to run evaluation on the val_set
    logging_first_step=False, # Whether to log also the very first training step to wandb
    load_best_model_at_end=True, # Whether to load the best model found at each evaluation.
    metric_for_best_model="loss", # Use loss to evaluate best model.
    greater_is_better=False # Best model is the one with the lowest loss, not highest.
    
)


TypeError: ignored

In [ ]:
!pip install torch==1.9.0+cu102 -f https://download.pytorch.org/whl/cu102/torch_stable.html


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu102 (from versions: 1.11.0, 1.11.0+cu102, 1.12.0, 1.12.0+cu102, 1.12.1, 1.12.1+cu102, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.0+cu102


In [ ]:
!pip install torch==1.9.0+cu102 torchvision==0.10.0+cu102 torchaudio===0.9.0 -f https://download.pytorch.org/whl/cu102/torch_stable.html



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/cu102/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu102 (from versions: 1.11.0, 1.11.0+cu102, 1.12.0, 1.12.0+cu102, 1.12.1, 1.12.1+cu102, 1.13.0, 1.13.1, 2.0.0, 2.0.1)
ERROR: No matching distribution found for torch==1.9.0+cu102


In [ ]:
import os
os.environ['TORCH_USE_CUDA_DSA'] = '1' #enable Device-side assertions debugging tool 
os.environ["CUDA_LAUNCH_BLOCKING"] = "1" # enable synchronous CUDA kernel error reporting

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

RuntimeError: ignored

In [ ]:
Trainer()

RuntimeError: ignored

In [ ]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

trainer.train()
trainer.save_model(output_dir + '/model')

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


RuntimeError: ignored

In [ ]:
import numpy as np

In [ ]:
model_dir = '/content/drive/My Drive/dir/model'
output_dir = '/content/drive/My Drive/dir'


model = T5ForConditionalGeneration.from_pretrained(model_dir)

pred_args = TrainingArguments(
    output_dir=output_dir,
    per_device_eval_batch_size=8,
    remove_unused_columns=True,
    eval_accumulation_steps=1
)

trainer = Trainer(model=model, args=pred_args)

preds, labels, *_ = trainer.predict(val_dataset)
preds_tokens = np.argmax(preds[0], axis=2)

decoded_sources = []
for row in val_dataset:
    decoded_sources.append(tokenizer.decode(row['input_ids']))

decoded_preds = [tokenizer.decode(pred) for pred in preds_tokens]
decoded_labels = [tokenizer.decode(label) for label in labels]

output = pd.DataFrame({'Source Text': decoded_sources, 'Target Text': decoded_labels, 'Generated Text': decoded_preds})
output.to_excel(output_dir + "/predictions.xlsx")

loading configuration file /content/drive/My Drive/dir/model/config.json
Model config T5Config {
  "_name_or_path": "t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
 

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("input:")
input_text = input()

with torch.no_grad():
    tokenized_text = tokenizer(input_text, truncation=True, padding=True, return_tensors='pt')

    source_ids = tokenized_text['input_ids'].to(device, dtype = torch.long)
    source_mask = tokenized_text['attention_mask'].to(device, dtype = torch.long)

    generated_ids = model.generate(
        input_ids = source_ids,
        attention_mask = source_mask, 
        max_length=512,
        num_beams=5,
        repetition_penalty=1, 
        length_penalty=1, 
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    pred = tokenizer.decode(generated_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)

print("\noutput:\n" + pred)

input:
what language am i speaking 


Generate config GenerationConfig {
  "_from_model_config": true,
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.28.1"
}




output:
Cutty '
